### 1. Configuraçoes Iniciais

In [103]:
import requests
import pandas as pd
import numpy as np
import os

Importando os dados do Fundo explorer

In [180]:
url = 'https://www.fundsexplorer.com.br/ranking'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]

Exibindo os dados

In [181]:
df.sort_values('Código do fundo', inplace=True) #classifica os dados utilizando o código do fundo
display(df) #imprime o DF


,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,...,Patrimônio Líq.,VPA,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos
234,AAZQ11,NaN,"R$ 9,64",42651.0,"R$ 0,14","1,46%","4,39%","0,00%","0,00%","1,46%",...,"R$ 230.646.828,88","R$ 9,60",100.0,"1,46%","-0,15%","1,30%","4,25%",NaN,NaN,0
186,ABCP11,Shoppings,"R$ 68,63",441.0,"R$ 0,50","0,73%","2,29%","4,59%","9,26%","0,76%",...,"R$ 432.278.740,15","R$ 91,80",75.0,"0,54%","0,11%","0,65%","2,52%","7,50%",NaN,1
318,AFHI11,Títulos e Val. Mob.,"R$ 94,28",8206.0,"R$ 1,10","1,15%","3,45%","6,61%","13,90%","1,15%",...,"R$ 286.309.475,82","R$ 94,82",99.0,"1,01%","-0,94%","0,06%","5,00%",NaN,NaN,0
285,AGRX11,NaN,"R$ 10,91",5306.0,"R$ 0,18","1,66%","4,82%","8,31%","0,00%","1,61%",...,"R$ 65.947.420,51","R$ 10,25",106.0,"1,76%","-0,49%","1,26%","2,84%",NaN,NaN,0
144,AIEC11,Lajes Corporativas,"R$ 61,99",7736.0,"R$ 0,77","1,28%","3,77%","7,02%","12,84%","1,26%",...,"R$ 455.490.836,76","R$ 94,40",66.0,"0,82%","0,19%","1,01%","1,20%","0,00%",NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,XPPR11,Outros,"R$ 23,79",14482.0,"R$ 0,10","0,43%","1,15%","3,89%","8,21%","0,38%",...,"R$ 523.050.642,76","R$ 71,49",33.0,"0,14%","3,00%","3,14%","-3,31%","46,00%","0,00%",4
141,XPSF11,Outros,"R$ 7,32",58976.0,"R$ 0,08","1,08%","3,26%","6,36%","12,27%","1,09%",...,"R$ 350.950.564,26","R$ 8,10",90.0,"0,94%","0,51%","1,45%","-3,30%",NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 8,82",2223.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 27.688.895,17","R$ 13,26",67.0,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
20,YUFI11B,Residencial,NaN,NaN,"R$ 0,63","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 50.141.745,85","R$ 101,78",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### 2. Limpeza e Formatação de Dados

In [106]:
#df.isna().sum() #mostra os dados NA

#### Dados categóricos

In [107]:
categorical_columns = ['Código do fundo','Setor'] #cria uma variavel carregando as colunas categoricas
idx = df[df['Setor'].isna()].index #mostra quais linhas possuem dados NA
#print(df[categorical_columns].isna().sum()) #mostra a soma de todas as linhas com NA
df.drop(idx, inplace=True) #apaga estas linhas
#df[categorical_columns].isna().sum() #mostra a soma de todas as linhas com NA
df[categorical_columns] = df[categorical_columns].astype('category') # define as colunas ocmo categoricas
#df.info()

#### Dados Numéricos

In [108]:
col_floats = list(df.iloc[:,2:-1].columns) #Seleciona da terceira até a penultima linha 
#print(col_floats)
df[col_floats] = df[col_floats].fillna(value=0) #substitui os dados NA por zero
df[col_floats] #valida se existe ainda dados nulos
df[col_floats] = df[col_floats].applymap(lambda x: str(x).replace('R$', '').replace('.0','').replace('.','').replace('%','').replace(',','.')) #limpa os dados removendo caracteres inuteis
df[col_floats] = df[col_floats].astype('float') #define as colunas como float
#df.info()
#df.describe()

##### Dados de P/VPA tem atributos infinitos e está em uma escala diferente

In [109]:
df = df.replace([np.inf, -np.inf], np.nan)  # Converter infinitos para NaN
df = df.dropna() #apaga os dados NA
df['P/VPA'] = df['P/VPA']/100 #convertendo P/VPA para
#df.describe()
#df

### 3. Analises (DF)

In [176]:
indicadores = ['Código do fundo','Setor', 'Preço Atual','DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']
indicadoresf = ['DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']

#### Média por Setor

In [111]:
#df.columns #exibe o nome de cabeçalho de todas as colunas do data set DF
df_aux = df[indicadores] #filtra o df pelos  cabeçalhos
media_setor = df_aux.groupby('Setor')[indicadoresf].agg(['mean']) #calcula a media dos indicadores agrupado por setor
media_setoru = media_setor.loc['Hotel', ('P/VPA', 'mean')]

#### Criando uma função com uma estratégia para oportunidades do mercado

In [124]:
def oportunidade_media_setor(df, setor, label_setor='Setor'):
    
    media_setor = df.groupby('Setor')[indicadoresf].agg(['mean']) #calcula a média dos indicadores do setor escolhido
    
    df_setor = df[df[label_setor].isin([setor])]
    
    filter_ = \
            (df_setor['Quantidade Ativos'] > 1) &\
            (df_setor['Liquidez Diária'] > 1) &\
            (df_setor['P/VPA'] < 2.0) &\
            (df_setor['DY (12M) Acumulado'] > media_setor.loc[setor, ('DY (12M) Acumulado','mean')]) 
            
    print('média do setor Yield: {}'.format(media_setor.loc[setor, ('DY (12M) Acumulado','mean')]))
    print('média do setor p/VPA: {}'.format(media_setor.loc[setor, ('P/VPA','mean')]))
    print('média do setor Ativos: {}'.format(media_setor.loc[setor, ('Quantidade Ativos','mean')]))
    
    return df_setor[filter_]

In [119]:
#list(df['Setor'].unique()) #mostra os setores do filtro selecionado

In [ ]:
dataframe = df_aux
setorF = 'Híbrido'
oportunidade = oportunidade_media_setor(dataframe, setorF)
oportunidade.sort_values('DY (12M) Acumulado', ascending=False, inplace=True)
oportunidade

### 4. Analises (FIIs Pessoais)

#### 4.1 Importacao de Tickets via CSV

In [ ]:
os.chdir("C:/Users/luis-/projetos/Python/analise-acoes/") #selecionando o diretorio do arquivo CSV a ser lido
tickets = pd.read_csv("./tickets.csv", sep = ',') #importando dados de um csv utilizando o pandas
tickets = tickets['Ativo'].tolist() #convertendo o data frame para lista
filtro = df.iloc[:, 0].isin(tickets) #filtando os dados utilizando as acoes do arquivo csv
df_filtro = df.loc[filtro]
display(df_filtro)

In [179]:
df_auxp = df_filtro[indicadores] #filtra o df pelos  cabeçalhos
filter_ = (df_aux['P/VPA'] < 1.01) & (df_aux['P/VPA'] > 0.85)
df_auxp = df_auxp[(filter_)].sort_values(['P/VPA'])
display(df_auxp)

C:\Users\luis-\AppData\Local\Temp\ipykernel_18588\391286116.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_auxp = df_auxp[(filter_)].sort_values(['P/VPA'])


,Código do fundo,Setor,Preço Atual,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
308,BRCO11,Logística,108.50,7.85,0.0,0.00,0.89,10,29879.0
165,XPLG11,Logística,105.19,8.86,8.8,1.20,0.92,13,25809.0
86,PVBI11,Lajes Corporativas,96.21,7.55,0.1,0.00,0.94,4,25668.0
70,KNRI11,Híbrido,153.49,7.80,2.0,6.72,0.96,20,23916.0
323,BTLG11,Logística,97.99,9.10,0.0,2.00,0.99,16,37684.0
107,HGRU11,Híbrido,124.17,9.77,0.0,0.00,1.00,17,25568.0


In [161]:
list(df_auxp['Setor'].unique()) #mostra os setores do filtro selecionado

['Logística', 'Híbrido', 'Lajes Corporativas', 'Títulos e Val. Mob.', 'Outros']

In [ ]:
oportunidade = oportunidade_media_setor(df_auxp, setor='Lajes Corporativas')
oportunidade.sort_values('DY (12M) Acumulado', ascending=False, inplace=True)
oportunidade